In [97]:
import os
new_directory = "/home/jovyan/work/ConsoleHermes"
os.chdir(new_directory)

from multiprocessing import Process
from matplotlib import pyplot
from scipy.optimize import curve_fit
import numpy
import ConsoleHermes.data_process as dp
import ConsoleHermes.plot_functions as pf
import ConsoleHermes.curve_functions as cf
import ConsoleHermes.covid as api



def getMergedWaves(FileName, ChunckSize, WaveOffset, MovingAverageIndex, GeneralPredictiveCapacityInDays, CurveModellingFunction=cf.Richards):
    count_results, date_results = api.getData(FileName)

    
    tmpDataTotalCases, tmpDataDailyCases, DataChunckSize_Cases = dp.PrepareDataToPlot(count_results, ChunckSize)
    #=======================================================================
    DataDailyCases = DataChunckSize_Cases
    DataTotalCases= dp.GenerateTotalCasesDataFromDailyCases(DataDailyCases)
    BestChunckSize = dp.GetBestChunckSize2(tmpDataDailyCases)
    #=======================================================================
    TotalDataLen = len(DataTotalCases)
    TotalDataInXAxis = GeneralPredictiveCapacityInDays + TotalDataLen
    #---------------------------------------------------
    PeakDays,PeakValues = dp.FindPeaks(DataDailyCases,WaveOffset) # 2 é o melhor parametro para casos em chuncks de semandas (7 dias) assim considera-se 2 semanas para confirmar o pico.
    WavesLimit = dp.BestFitWavesLimitDefination2(DataDailyCases,PeakDays,MovingAverageIndex)
    #---------------------------------------------------
    if len(PeakDays) > len(WavesLimit):
        WavesLimit.append(len(DataDailyCases))
    #---------------------------------------------------
    DetectedWaves = len(WavesLimit)
    #---------------------------------------------------
    WavesData = list(numpy.empty(len(WavesLimit)))
    WavesLength = list(numpy.empty(len(WavesLimit)))
    #---------------------------------------------------
    GraphicalTotalCasesWave = list(numpy.empty(len(WavesLimit)))
    GraphicalDailyCasesWave = list(numpy.empty(len(WavesLimit)))
    FuturePredictionRangeWave = list(numpy.empty(len(WavesLimit)))

    IndividualDataX = list(numpy.empty(len(WavesLimit)))
    IndividualDataY = list(numpy.empty(len(WavesLimit)))

    
    DataIgnore = 0
    PreviousWaveLimit = 0
    for i in range(0,DetectedWaves):
        WavesData[i] = dp.CopyListArray(DataTotalCases[:WavesLimit[i]])
        WavesLength[i] = len(WavesData[i]) - DataIgnore
        PreviousWaveLimit = WavesLimit[i]
        DataIgnore = len(WavesData[i])
        FuturePredictionRangeWave[i] = TotalDataInXAxis - WavesLimit[i]
    #---------------------------------------------------
    #iniciando a primeira onda! 
    #---------------------------------------------------

    ColorsData = ['red','green','blue','orange','pink','purple','mustard','beige']
    DataToPlot = DataDailyCases
    #---------------------------------------------------    
    LastWaveValue = 0
    LastDailyWaveValue = 0
    #---------------------------------------------------
    for i in range(0,DetectedWaves):
        Kik_K = 15000000
        filler = [0,0,0,0,0,0,0,0,0,0]
        Datay   = dp.CopyListArray(WavesData[i])
        if (i>0):
            #---------------------------------------------------
            RealDataWave = dp.CopyListArray(dp.GenerateDailyCasesDataFromTotalCases(Datay))
            SimDataWave  = dp.CopyListArray(GraphicalDailyCasesWave[i-1][0:len(GraphicalDailyCasesWave[i-1])-1])
            #---------------------------------------------------
            NewWaveDailyCases = dp.PreparedDataToPerformGrowthFunction(SimDataWave,RealDataWave,WavesLimit[i-1])
            NewWaveTotalCases = dp.GenerateTotalCasesDataFromDailyCases(NewWaveDailyCases)
            #---------------------------------------------------
            Datay   = dp.CopyListArray(NewWaveTotalCases)
        #---------------------------------------------------   
        Datax   = list (range(1,len(Datay)+1))
        GraphicalTotalCasesWave[i] = numpy.zeros(len(WavesData[i]), dtype='int')
        GraphicalDailyCasesWave[i] = numpy.zeros(len(WavesData[i]), dtype='int')
        try:
            popt, pcov = curve_fit(CurveModellingFunction, Datax, Datay, p0=[Kik_K , 1 , 0.5, 1], maxfev=800000)
            K, A, r, s = popt
            Kik_K = K
            tmpDataX = numpy.array(Datax, dtype='int')
            tmpDataY = Datay
            #---------------------------------------------------
            GraphicalTotalCasesWave[i] = dp.GenerateTotalCasesDataCurveFitting(range(1,len(WavesData[i])+1), K, A, r, s,FuturePredictionRangeWave[i], CurveModellingFunction)
            GraphicalDailyCasesWave[i] = dp.GenerateDailyCasesDataFromTotalCases(GraphicalTotalCasesWave[i])
        except:
            # print(error)
            # print("Not enought data for wave", i)
            continue
        #---------------------------------------------------
        strlabel="Wave " +str(i+1)
        DataToPlotTMP = dp.CopyListArray(DataToPlot)
        DataToPlot = dp.CopyListArray(GraphicalDailyCasesWave[i][0:len(GraphicalDailyCasesWave[i])-1])
        IndividualDataY[i] = DataToPlot
        #---------------------------------------------------
        if (i==0):
            XAxis = range(1,len(DataToPlot )+1)
            YAxis = DataToPlot
        else:
            DataToPlot[0] = DataToPlot[1] 
            DataToPlot = numpy.insert(DataToPlot,numpy.zeros((WavesLimit[i-1],), dtype=int),0)
            YAxisTMP = dp.CopyListArray(DataToPlot[WavesLimit[i-1]:len(DataToPlot)-WavesLimit[i-1]])
            YAxis = YAxisTMP
            XAxis = range(WavesLimit[i-1],len(DataToPlot)-WavesLimit[i-1]+0)
    #====================================================================================================
    # Total Cases Wave Plotting
    #====================================================================================================
    ColorsData = ['red','green','blue','orange','pink','purple','mustard','beige']
    DataToPlot = DataTotalCases
    BarXAxius = range(1,len(DataToPlot)+1)
    BarYAxius = dp.CopyListArray(DataToPlot)
    #---------------------------------------------------        
    LastWaveValue = 0
    PlotedData = [1]
    PlotedPredictions = [1]
    FlatPredictions = []
    Waves_R_Squared = []
    #---------------------------------------------------
    for i in range(0,DetectedWaves):
        hasLength = hasattr(GraphicalTotalCasesWave[i], '__len__')
        if (hasLength == False):
            break
        strlabel="Wave " +str(i+1)
        DataToPlotTMP = dp.CopyListArray(DataToPlot)
        DataToPlot = dp.CopyListArray(GraphicalTotalCasesWave[i][0:len(GraphicalTotalCasesWave[i])-1])
        if (i==0):
            YAxis = DataToPlot
            XAxis = range(1,len(YAxis)+1)

            RealData = numpy.array(BarYAxius[:WavesLimit[i]-1], dtype='int')
            PredictedData = numpy.array(YAxis[:WavesLimit[i]-1], dtype='int')
            FlatPredictions.append(PredictedData)
            
            r_squared = dp.GetRSquared(RealData, PredictedData)
            Waves_R_Squared.append(r_squared)
        else:
            DataToPlot = numpy.insert(DataToPlot,numpy.zeros((WavesLimit[i-1],), dtype=int),0)
            for q in range (1,(len(DataToPlot) - WavesLimit[i-1])):
                DataToPlot[q] = DataToPlot[q] + DataToPlotTMP[q]
            #---------------------------------------------------            
            YAxis = dp.CopyListArray(DataToPlot[WavesLimit[i-1]:len(DataToPlot)-WavesLimit[i-1]])
            RealData = numpy.array(BarYAxius[WavesLimit[i-1]:WavesLimit[i]+0], dtype='int')
            PredictedData = numpy.array(YAxis[:WavesLength[i]], dtype='int')
            FlatPredictions.append(PredictedData)
            #---------------------------------------------------
            r_squared = dp.GetRSquared(RealData, PredictedData)
            Waves_R_Squared.append(r_squared)
            #---------------------------------------------------
            XAxis = range(WavesLimit[i-1],len(DataToPlot)-WavesLimit[i-1]+0)
    #---------------------------------------------------
    ProcessedPredictions = numpy.array([item for sublist in FlatPredictions for item in sublist])
    DescriptedData = numpy.array(BarYAxius[:len(BarYAxius)-1])
    WavesRSquaredAverage = numpy.mean(Waves_R_Squared)
    return ProcessedPredictions, DescriptedData, WavesRSquaredAverage

In [98]:
#===================================================================================
# Perform Best Parameters Analysis - (Search for best parameters values)
#===================================================================================
#PARAMETERS:
#   DataSetFileName........: Complete name of dataser file, with extension (.csv)
#                            Example: "Covid_TotalCases_Brazil.csv"
#   MaxChunckSize..........: Maximum sife for chuck data
#   MaxWaveOffset..........: Maximum sife for waves offset
#   MaxMovingAverageIndex..: Maximum sife for Moving Average Index 
#RETURN:
#   None.
#        This function perform search for best parametes values and writes results on 
#        specif file. 
#-----------------------------------------------------------------------------------
#Call...........: PerformBestParametersAnalysis('Covid_TotalCases_Brazil.csv',4,4,4)
#-----------------------------------------------------------------------------------
def PerformBestParametersAnalysis(DataSetFileName, MaxChunckSize, MaxWaveOffset, MaxMovingAverageIndex):
    CsvHeader = ['chunck_size', 'wave_offset', 'moving_average_index', 'r_squared']
    OutputFileName ="Best_Parameters_" + DataSetFileName
    with open('Files/BestParameters/'+OutputFileName, 'w') as FileNamecsv:
        writer = csv.writer(FileNamecsv)
        writer.writerow(CsvHeader)
        print("Starting Analyse for:", DataSetFileName)
        for ChunckSize in range(3, MaxChunckSize+1):
            print('-------------Analysing ChunckSize:', ChunckSize)
            for WaveOffset in range(3, MaxWaveOffset+1):
                #print('--------------------------Analysing WaveOffset:', WaveOffset)
                for MovingAverage in range(3, MaxMovingAverageIndex+1):
                    #print('---------------------------------------Analysing MovingAverage:', MovingAverage)
                    try:
                        ProcessedPredictions, DescriptedData, WavesRSquaredAverage = getMergedWaves(DataSetFileName,ChunckSize, WaveOffset, MovingAverage, 10, cf.Richards)
                        r_squared = dp.GetRSquared(ProcessedPredictions, DescriptedData)
                        if (r_squared < 1):
                            writer.writerow([ChunckSize, WaveOffset, MovingAverage, r_squared])
                    except:
                        writer.writerow([ChunckSize, WaveOffset, MovingAverage, 'FAILED'])

In [99]:
thread1 = Process(target=PerformBestParametersAnalysis, args=('',4, 4, 4))
#thread1.start()

In [107]:
import time
import warnings
import sys
from scipy.optimize import OptimizeWarning
warnings.filterwarnings("ignore", category=RuntimeWarning)
sys.warnoptions.append("ignore")
warnings.filterwarnings("ignore", message="Covariance of the parameters could not be estimated", category=OptimizeWarning)

inicio = time.time()  # Captura o tempo atual antes de chamar a função
#====================================================================
#====================================================================
PerformBestParametersAnalysis('Covid_TotalCases_Brazil.csv',7,15,20)
#====================================================================
#====================================================================
fim = time.time()     # Captura o tempo atual após a função ser executada
tempo_execucao = fim - inicio
if tempo_execucao > 60:
    tempo_execucao_minutos = tempo_execucao / 60
    print("Tempo de execução:", tempo_execucao_minutos, "minutos")
else:
    print("Tempo de execução:", tempo_execucao, "segundos")


Starting Analyse for: Covid_TotalCases_Brazil.csv
-------------Analysing ChunckSize: 3
-------------Analysing ChunckSize: 4
-------------Analysing ChunckSize: 5
-------------Analysing ChunckSize: 6
-------------Analysing ChunckSize: 7
Tempo de execução: 7.443489464124044 minutos
